In [1]:
#QRM gate counts
#June 27th 2023
import math
import numpy as np
from qrm_counts import *
r, m = 7, 10

print(naive_CX_count(r, m))
print(std_CX_count(r, m))
print(rec_CX_count(r, m))

param_list = [(1, 3), (2, 4), (3, 4), (2, 5), (3, 5), (4, 5), (5, 5), (3, 6), (4, 6), (5, 6), (3, 7), (4, 7), (5, 7), (6, 7), (4, 8), (5, 8), (6, 8), (7, 8), (8, 10), (9, 10)]

print(rec_CX_count_assym(2, 5, 2, 5))

for params in param_list:
    r, m = params
    print('\n', r, m)
    print(naive_CX_count(r, m))
    print(std_CX_count(r, m))
    print(rec_CX_count(r, m))

57880
20664
5096
66

 1 3
16
12
10

 2 4
61
53
30

 3 4
65
29
32

 2 5
176
112
66

 3 5
206
150
78

 4 5
211
61
80

 5 5
211
0
80

 3 6
614
470
176

 4 6
659
367
190

 5 6
665
125
192

 3 7
1744
960
372

 4 7
1989
1389
430

 5 7
2052
840
446

 6 7
2059
253
448

 4 8
5821
3933
930

 5 8
6213
3605
1004

 6 8
6297
1857
1022

 7 8
6305
509
1024

 8 10
58015
8627
5118

 9 10
58025
2045
5120


In [1]:
import stim
circuit = stim.Circuit()

# First, the circuit will initialize a Bell pair.
circuit.append("H", [0])
circuit.append("CNOT", [0, 1, 3, 4])

# Then, the circuit will measure both qubits of the Bell pair in the Z basis.
circuit.append("M", [0, 1])

# Let's see the circuit's representation using stim's circuit language:
print(repr(circuit))



circuit = stim_CNOT_list(circuit, [0, 1], [2, 3])
circuit = stim_H_list(circuit, [0, 1])
print(circuit)

stim.Circuit('''
    H 0
    CX 0 1 3 4
    M 0 1
''')


NameError: name 'stim_CNOT_list' is not defined

In [25]:
from qrm_circuits import QRM_rec_circuit, tequila_to_stim

circ, info = QRM_rec_circuit(3, 4)

st_circuit= tequila_to_stim(circ)
st_circuit.append("DEPOLARIZE2", [0, 1, 2, 3], 0.5)

In [12]:
from qrm_circuits import QRM_rec_circuit, get_qubit_partition
from qrm_utils import get_qiskit_circuit

c, i = QRM_rec_circuit(3, 4, partitions=[2, [2], [2]], only_cnots=True)
print(get_qiskit_circuit(c))

                                                                            »
 q_0: ──■──────────────■────────────────────────■────────────────────────■──»
        │              │                        │                      ┌─┴─┐»
 q_1: ──┼────■─────────┼────■───────────────────┼───────────────────■──┤ X ├»
        │    │       ┌─┴─┐  │                   │                   │  └───┘»
 q_2: ──┼────┼────■──┤ X ├──┼───────────────────┼────■──────────────┼───────»
        │    │    │  └───┘┌─┴─┐                 │    │              │       »
 q_3: ──┼────┼────┼────■──┤ X ├─────────────────┼────┼──────────────┼────■──»
      ┌─┴─┐  │    │    │  └───┘                 │    │              │    │  »
 q_4: ┤ X ├──┼────┼────┼────■─────────■─────────┼────┼─────────■────┼────┼──»
      └───┘┌─┴─┐  │    │    │         │         │    │       ┌─┴─┐  │    │  »
 q_5: ─────┤ X ├──┼────┼────┼────■────┼─────────┼────┼────■──┤ X ├──┼────┼──»
           └───┘┌─┴─┐  │  ┌─┴─┐  │    │         │    │    │  └──

In [5]:
from qrm_counts import rec_CX_count_assym_punc
param_list = [(1, 3), (1, 4), (2, 5), (2, 6), (2, 7)]
for param in param_list:
    r, m = param
    a = rec_CX_count_assym_punc(r, m, r, m, state_prep=True)
    b = rec_CX_count_assym_punc(r, m, r, m, state_prep=False)
    print('r: {} m: {}, |0>: {} |+>: {}'.format(r, m, a, b))
    

r: 1 m: 3, |0>: 8 |+>: 8
r: 1 m: 4, |0>: 20 |+>: 20
r: 2 m: 5, |0>: 60 |+>: 60
r: 2 m: 6, |0>: 137 |+>: 137
r: 2 m: 7, |0>: 294 |+>: 294


In [1]:
from qrm_circuits import QRM_rec_punc_circuit, QRM_punc_std_circuit, canonical_CSS
from qrm_utils import get_qiskit_circuit, check_if_same_circuit
from qrm_matrices import get_QRM_punc_generator
import tequila

r, m = 1, 3
c, i = QRM_rec_punc_circuit(r, m)
#c2, i2 = QRM_punc_std_circuit(r, m)

G = get_QRM_punc_generator((r, m), (r, m))
c2, i = canonical_CSS(G, (2**m)-1)

print(get_qiskit_circuit(c), get_qiskit_circuit(c2))
tequila.simulate(c)
print(check_if_same_circuit(c, c2))

0 2
     ┌───┐                         
q_0: ┤ H ├──■─────────■────────────
     ├───┤  │         │            
q_1: ┤ H ├──┼────■────┼────■───────
     └───┘  │    │  ┌─┴─┐┌─┴─┐     
q_2: ───────┼────┼──┤ X ├┤ X ├─────
     ┌───┐  │    │  └───┘└───┘     
q_3: ┤ H ├──┼────┼────■─────────■──
     └───┘┌─┴─┐  │    │       ┌─┴─┐
q_4: ─────┤ X ├──┼────┼────■──┤ X ├
          └───┘┌─┴─┐┌─┴─┐  │  └───┘
q_5: ──────────┤ X ├┤ X ├──┼────■──
               └───┘└───┘┌─┴─┐┌─┴─┐
q_6: ────────────────────┤ X ├┤ X ├
                         └───┘└───┘
c: 7/══════════════════════════════
                                         ┌───┐                                                                 
q_0: ┤ H ├───────■────■───────────────────■────■────■────■────■────■────■──
     ├───┤       │    │                   │  ┌─┴─┐  │    │    │    │    │  
q_1: ┤ H ├──■────┼────┼────■─────────■────┼──┤ X ├──┼────┼────┼────┼────┼──
     └───┘┌─┴─┐┌─┴─┐  │    │         │    │  └───┘┌─┴─┐  │    │    │    │  
q_2:

In [5]:
print(tequila.simulate(c),'\n', tequila.simulate(c2))

+0.3536|0000000> +0.3536|0111100> +0.3536|1011010> +0.3536|1100110> +0.3536|1101001> +0.3536|1010101> +0.3536|0110011> +0.3536|0001111>  
 +0.3536|0000000> +0.3536|0111100> +0.3536|1101010> +0.3536|1010110> +0.3536|1011001> +0.3536|1100101> +0.3536|0110011> +0.3536|0001111> 


In [5]:
from qrm_circuits import QRM_rec_circuit, QRM_rec_assym_circuit
from qrm_utils import get_qiskit_circuit
from qrm_counts import rec_CX_count_assym, rec_CX_count
import tequila
r, m = 2, 4
c, i = QRM_rec_circuit(r, m, only_cnots=True)
print(get_qiskit_circuit(c))
print(len(c.gates))
print(rec_CX_count(r, m))
print(tequila.simulate(c))

                                                                       
 q_0: ──────────────────────■────────────────────────■────■────────────
                            │                        │  ┌─┴─┐          
 q_1: ──■───────────────────┼────■──────────────■────┼──┤ X ├──────────
        │                   │    │              │  ┌─┴─┐└───┘          
 q_2: ──┼────■──────────────┼────┼─────────■────┼──┤ X ├──■────────────
        │    │              │    │         │  ┌─┴─┐└───┘┌─┴─┐          
 q_3: ──┼────┼────■─────────┼────┼────■────┼──┤ X ├─────┤ X ├──────────
        │    │    │       ┌─┴─┐  │    │    │  └───┘     └───┘          
 q_4: ──┼────┼────┼────■──┤ X ├──┼────┼────┼─────────■────■────────────
        │    │    │    │  └───┘┌─┴─┐  │    │         │  ┌─┴─┐          
 q_5: ──┼────┼────┼────┼────■──┤ X ├──┼────┼────■────┼──┤ X ├──────────
        │    │    │    │    │  └───┘  │  ┌─┴─┐  │  ┌─┴─┐└───┘          
 q_6: ──┼────┼────┼────┼────┼────■────┼──┤ X ├──┼──┤ X ├──■─────

In [5]:
from qrm_circuits import QRM_rec_classical_circuit, QRM_rec_circuit
from qrm_utils import get_qiskit_circuit

c, i = QRM_rec_classical_circuit(1, 3)
print(get_qiskit_circuit(c))
print(len(c.gates))

                                   
q_0: ──■───────────────────■────■──
       │                   │  ┌─┴─┐
q_1: ──┼────■─────────■────┼──┤ X ├
       │    │         │  ┌─┴─┐└───┘
q_2: ──┼────┼────■────┼──┤ X ├──■──
       │    │    │  ┌─┴─┐└───┘┌─┴─┐
q_3: ──┼────┼────┼──┤ X ├─────┤ X ├
     ┌─┴─┐  │    │  └───┘     └───┘
q_4: ┤ X ├──┼────┼────■─────────■──
     └───┘┌─┴─┐  │    │       ┌─┴─┐
q_5: ─────┤ X ├──┼────┼────■──┤ X ├
          └───┘┌─┴─┐┌─┴─┐  │  └───┘
q_6: ──────────┤ X ├┤ X ├──┼────■──
               └───┘└───┘┌─┴─┐┌─┴─┐
q_7: ────────────────────┤ X ├┤ X ├
                         └───┘└───┘
c: 8/══════════════════════════════
                                   
11


In [2]:
from qrm_matrices import get_QRM_generator
from qrm_utils import *

def get_QRM_punc_generator(C1_params: tuple, C2_params: tuple):
    G2perp, G1q = get_QRM_generator(C1_params=C1_params, C2_params=C2_params)
    G2perpt = puncture_matrix(G2perp)
    G1qp = puncture_matrix(G1q)
    print(G2perpt)
    G2perpp = []
    for row in G2perpt:
        if np.sum(row) != len(row):
            G2perpp.append(row)
        else:
            G1qp.append(row)
    return (G2perpp, G1qp)

get_QRM_punc_generator((2, 4), (2, 4))

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1], [0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]]


([[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
  [0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
  [0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1],
  [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]],
 [[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1],
  [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1],
  [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1],
  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [4]:
#CSS encoders
import tequila
from qrm_circuits import QRM_std_circuit, QRM_rec_circuit
from qrm_utils import *
from qrm_circuits import QRM_std_circuit
from qrm_matrices import *

#need to generalize stuff
r, m = 2, 4
circuit, info = QRM_rec_circuit(r, m, partitions=[1, [1], [1]])
std_circuit, info2 = QRM_std_circuit(r, m)
qiskit_cir = get_qiskit_circuit(circuit)
print(info, qiskit_cir, circuit.depth, len(circuit.gates))
check_if_same_circuit(circuit, std_circuit)

None       ┌───┐                                                                 
 q_0: ┤ H ├─────────────────────────────────────■───────────────────■────■──
      ├───┤                                     │                   │  ┌─┴─┐
 q_1: ┤ H ├─────────────────■───────────────────┼────■─────────■────┼──┤ X ├
      ├───┤                 │                   │    │         │  ┌─┴─┐└───┘
 q_2: ┤ H ├─────────────────┼────■──────────────┼────┼────■────┼──┤ X ├──■──
      └───┘                 │    │              │    │    │  ┌─┴─┐└───┘┌─┴─┐
 q_3: ───────■──────────────┼────┼────■─────────┼────┼────┼──┤ X ├─────┤ X ├
      ┌───┐  │              │    │    │       ┌─┴─┐  │    │  └───┘     └───┘
 q_4: ┤ H ├──┼──────────────┼────┼────┼────■──┤ X ├──┼────┼─────────■────■──
      └───┘  │              │    │    │    │  └───┘┌─┴─┐  │         │  ┌─┴─┐
 q_5: ───────┼────■─────────┼────┼────┼────┼───────┤ X ├──┼────■────┼──┤ X ├
             │    │         │    │    │    │       └───┘┌─┴─┐  │  ┌─┴─┐

True

In [1]:
import tequila
from qrm_circuits import QRM_rec_circuit, QRM_std_circuit
only_cnots = False
circuit, info1 = QRM_rec_circuit(3, 4, only_cnots=only_cnots)
circuit_std, info2 = QRM_std_circuit(3, 4, only_cnots=only_cnots, transform_rows=False)
msg = tequila.QCircuit()

msg += tequila.gates.H(target=info1[0][0])
msg += tequila.gates.CX(target=info1[0][1], control=info1[0][0])
print(len(circuit_std.gates), len(circuit.gates))
from qrm_utils import check_if_same_circuit

check_if_same_circuit(msg + circuit, msg + circuit_std)

66 33


True

In [4]:
#generate check matrix and find the diagonalizing circuit
from qrm_matrices import *
r, m = (2, 3)
G = Grm(r, m)
H = Hrm(r, m)
#HCSS = 

import tequila
from tequila.grouping.binary_rep import BinaryPauliString, BinaryHamiltonian

H = Hqrm(1, 3)

bps = [BinaryPauliString(binary_vector=bv, coeff=1.0) for bv in H]
BH = BinaryHamiltonian(binary_terms=bps)

stuff = BH.get_qubit_wise()

ImportError: attempted relative import with no known parent package